## Introduction

The posterior probabilities computed by the `findr` function can be used as a graph structure prior in [Bayesian network](https://en.wikipedia.org/wiki/Bayesian_network) learning using the model introduced in the paper [High-dimensional Bayesian network inference from systems genetics data using genetic node ordering](https://doi.org/10.3389/fgene.2019.01196). This involves converting the pairwise posterior probabilities to a [directed acyclic graph (DAG)](https://en.wikipedia.org/wiki/Directed_acyclic_graph). 

[BioFindr][1] implements the original *"greedy edges"* algorithm where edges are added one-by-one in decreasing order of probability, and only if they do not create a cycle in the graph, using an incremental cycle detection algorithm. Two additional algorithms from the paper [Maximal acyclic subgraph optimization for gene regulatory networks](https://biblio.ugent.be/publication/8772612) are also implemented: the *heuristic sort* algorithm where vertices are sorted by their ratio of out-degree to in-degree, and edges are added only if their source vertex precedes their target vertex in the sorted list, and the  *greedy insertion* algorithm where vertices are iteratively inserted in the position in the current ordering that yields the maximum possible gain of edge weights, where the gain is counted as the difference between the sum of new edge weights included and the sum of old edge weights lost, and edges are counted only if their source vertex precedes their target vertex in the ordering.

## Set up the environment


In [ ]:
using DrWatson
quickactivate(@__DIR__)

using DataFrames
using Arrow
using GLMakie
using GraphMakie

using BioFindr

## Load data

For illustration we use the GEUVADIS microRNA data:


In [ ]:
dm = DataFrame(Arrow.Table(datadir("exp_pro","findr-data-geuvadis", "dm.arrow")));
dgm = DataFrame(Arrow.Table(datadir("exp_pro","findr-data-geuvadis", "dgm.arrow")));
dpm = DataFrame(SNP_ID = names(dgm), GENE_ID=names(dm)[1:ncol(dgm)]);

## Run BioFindr

We perform causal inference to compute posterior probabilities from all microRNAs with an eQTL to the total set of microRNAs:


In [ ]:
dP = findr(dm, dgm, dpm; FDR=0.25)

To construct a DAG from the DataFrame `dP` with posterior probabilities using the default *"greedy edges"* algorithm, run:


In [ ]:
G, name2idx = dagfindr!(dP);

The `!` in the `dagfindr!` function name indicates that the function modifies its input argument. We can see that `dP` indeed contains some new columns:


In [ ]:
dP

The **Source_idx** and **Target_idx** are numerical IDs for the **Source** and **Target** nodes, respectively, and the **inDAG_greedy_edges** indicates whether the edge represented by a row of `dP` in included in the output DAG `G`. The mapping from node names to IDs is also returned as a [dictionary object](https://docs.julialang.org/en/v1/base/collections/#Dictionaries) `name2idx`.


In [ ]:
name2idx

The output `G` is a [directed graph object](https://juliagraphs.org/Graphs.jl/stable/core_functions/module/#Graphs.DiGraph) from the [Graphs package](https://juliagraphs.org/Graphs.jl/stable/):


In [ ]:
G

This is a fairly simple datastructure, which only supports numerical node IDs, hence the need to create the `name2idx` map. One useful thing one can do with a `Graph` object is to draw it:


In [ ]:
graphplot(G)

For more details, see [the documentation](https://juliagraphs.org/Graphs.jl/dev/first_steps/plotting/).

To run the `dagfindr!` function with the other DAG construction algorithms mentioned in the Introduction,


In [ ]:
G, name2idx = dagfindr!(dP; method="heuristic sort");
G, name2idx = dagfindr!(dP; method="greedy insertion");

The results of these `dagfindr!` calls are added to `dP` allowing easy comparison of the methods:


In [ ]:
dP

[1]: https://github.com/tmichoel/BioFindr.jl